In [11]:
! uv pip install cryptography

Using Python 3.12.9 environment at: C:\Users\KarthikeyanVK\.conda\envs\byteai
Audited 1 package in 103ms


In [12]:
import base64
import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

def get_key_from_password(password: str, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100_000,
    )
    return base64.urlsafe_b64encode(kdf.derive(password.encode()))


In [13]:
from cryptography.fernet import Fernet
import json, os

def encrypt_file(file_path, output_dir, fake_name, password):
    with open(file_path, 'rb') as f:
        file_data = f.read()

    salt = os.urandom(16)
    key = get_key_from_password(password, salt)
    print(key)
    fernet = Fernet(key)

    metadata = {
        "original_filename": os.path.basename(file_path)
    }
    encrypted_metadata = fernet.encrypt(json.dumps(metadata).encode())
    encrypted_data = fernet.encrypt(file_data)

    os.makedirs(output_dir, exist_ok=True)
    full_output_path = os.path.join(output_dir, fake_name)

    with open(full_output_path, 'wb') as out:
        out.write(salt)  # 16 bytes
        out.write(len(encrypted_metadata).to_bytes(4, 'big'))
        out.write(encrypted_metadata)
        out.write(encrypted_data)


In [14]:
def decrypt_file(encrypted_path, output_dir, password):
    with open(encrypted_path, 'rb') as f:
        salt = f.read(16)
        key = get_key_from_password(password, salt)
        fernet = Fernet(key)

        meta_len = int.from_bytes(f.read(4), 'big')
        encrypted_metadata = f.read(meta_len)
        encrypted_data = f.read()

    metadata = json.loads(fernet.decrypt(encrypted_metadata))
    original_filename = metadata['original_filename']
    decrypted_content = fernet.decrypt(encrypted_data)

    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, original_filename)
    with open(output_path, 'wb') as f:
        f.write(decrypted_content)


In [15]:
encrypt_file(
    "./prosat-slides-ideation.pdf",
    "./encrypt/",
    "masked-invoice.pdf",  # this will be visible
    "password123"
)

decrypt_file(
    "./encrypt/masked-invoice.pdf",
    "./decrypted/",
    "password123"
)


b'W2-He2-yljyzrUVNxxAARpQj8r3BBhN_Bom17KWnhL4='
